# **Ejercicio 1) Sistema de ecuaciones lineales**

**Se quiere obtener una corriente de producto a partir de la mezcla de corrientes provenientes de distintos procesos. <br>La corriente buscada debe tener un caudal de 100.000 kg/h y la siguiente composición: 56% metanol, 21% etanol, 2% propanol y 21% de agua. <br> Se conocen las composiciones de todas las corrientes que ingresan al tanque de mezclado (F1, F2, F3, y F4):**

<img src="TPI2C2021Tanque.jpg" width=400 height=400 />

In [1]:
#Importamos los módulos a utilizar
#-------------------------------------------------------------------------------------------------------------------------#

import pandas as pd
import numpy as np

In [2]:
Compuestos = pd.Series(['Metanol', 'Etanol', 'Propanol', 'Agua'])
F1_composicion = pd.Series([0.83, 0, 0, 0.17])
F2_composicion = pd.Series([0, 0.61, 0, 0.39])
F3_composicion = pd.Series([0, 0, 0.90, 0.10])
F4_composicion = pd.Series([0.55, 0.22, 0, 0.20])
P_composicion = pd.Series([0.56, 0.21, 0.02, 0.21])

pd.DataFrame({ 'Compuestos': Compuestos, 'F1': F1_composicion, 'F2': F2_composicion, 'F3': F3_composicion, 'F4': F4_composicion, 'P': P_composicion })

Compuestos    F1    F2   F3    F4     P
0    Metanol  0.83  0.00  0.0  0.55  0.56
1     Etanol  0.00  0.61  0.0  0.22  0.21
2   Propanol  0.00  0.00  0.9  0.00  0.02
3       Agua  0.17  0.39  0.1  0.20  0.21

**1. Calcular las corrientes de entrada (F1, F2, F3 y F4) para obtener la corriente de producto buscada. Para realizar los cálculos elegir un método de resolución de SEL. Justificar la elección de cada método y comparar resultados.**

Realizando los balances de materia por componente se obtiene el siguiente sistema de ecuaciones: 

$BM_{general}) F_1 + F_2 + F_3 + F_4 = P$

$BM_{Metanol}) F_1 * 0.83 + F_4 * 0.55 = P * 0.56$

$BM_{Etanol}) F_2 * 0.61 + F_4 * 0.22 = P * 0.21$

$BM_{Propanol}) F_3 * 0.9 = P * 0.02$

$BM_{Agua}) F_1 * 0.17 + F_2 * 0.39 + F_3 * 0.1 + F_4 * 0.20 = P * 0.56$

Obtenemos un sistema de 5 ecuaciones con 4 incognitas, asique podemos eliminar una ecuacion que seria linealmente dependiente.

In [3]:
A = np.array([[0.83, 0, 0, 0.55],
       [0, 0.61, 0, 0.22],
       [0, 0, 0.9, 0],
       [0.17, 0.39, 0.1, 0.20]])

B = np.array([56000, 21000, 2000, 21000])

#### Luego de un primer análisis de A, llegamos a la conclusión de que no era ni definida positiva, ni tampoco diagonal dominante/Estrictamente diagonal dominante, lo cual redujo nuestras opciones para resolver el SEL propuesto.

<img src="AutoValoresWolfram.png" width=400 height=400 />

In [21]:
#Una forma alternativa para comprobar que A no es definida positiva es utilizar el método de Cholesky
#-------------------------------------------------------------------------------------------------------------------------#
def factorizarCholesky(A):
    n = np.shape(A)[0]
    S = np.zeros((n,n))
    for i in range (0,n):
        S[i,i] = A[i,i]  
        for k in range(0,i):
            S[i,i] += - S[i,k]**2
        S[i,i] = np.sqrt(S[i,i])   
        for j in range (i+1,n):     
            S[j,i] = A[j,i]
            for k in range(0,i):           
                S[j,i] += - S[j,k]*S[i,k]                  
            S[j,i] = S[j,i] / S[i,i]               
    return S

In [23]:
#El método tiene como condición necesaria que A sea definida positiva
#Esto se debe a que su formulación parte de descomponer a A como (S)*D*(S**T) (Condición necesaria, A simétrica)
#Y luego eleva a D a la potencia fraccionaria 1/2 (Raíz cuadrada de D). Si D no tiene autovalores > 0 devuelve error
#-------------------------------------------------------------------------------------------------------------------------#
#print(A)
factorizarCholesky(A)

<ipython-input-21-25ab4c1639a6>:10: RuntimeWarning: invalid value encountered in sqrt
  S[i,i] = np.sqrt(S[i,i])


array([[0.91104336, 0.        , 0.        , 0.        ],
       [0.        , 0.78102497, 0.        , 0.        ],
       [0.        , 0.        , 0.9486833 , 0.        ],
       [0.18659924, 0.49934383, 0.10540926,        nan]])

In [4]:
#Método de Doolittle (Programado por el grupo)
#En vez de utilizar un enfoque aprovechando las herramientas de Numpy, se optó por un enfoque con operaciones matriciales
#-------------------------------------------------------------------------------------------------------------------------#

def startMatrix(A):
    m = []
    rangeMatrix = len(A[0])
    for i in range(rangeMatrix):
        m.append([])
        for j in range(rangeMatrix):
            m[i].append(0)
    return m
 
def summDiagonalU(upperMatrix,lowerMatrix,i):
    summ = 0
    for k in range(0,i):
        summ += lowerMatrix[i][k] * upperMatrix[k][i]
        #print("k es igual a:", k)
        #print("Summ es = a: ", summ)
    return summ


def summU(upperMatrix, lowerMatrix, i, j):
    summ = 0  
    for k in range(0,i):
        summ += lowerMatrix[i][k] * upperMatrix[k][j]
    return summ
        
        
def summL(upperMatrix, lowerMatrix, i, j):
    summ = 0
    for k in range(0,i):
        summ += lowerMatrix[j][k] * upperMatrix[k][i]
    return summ

def doolittle(A):
    rangeMatrix = len(A[0])
    upperMatrix = startMatrix(A)
    lowerMatrix = startMatrix(A)
    for i in range(rangeMatrix):
        lowerMatrix[i][i] = 1
        upperMatrix[i][i] = A[i][i] - summDiagonalU(upperMatrix, lowerMatrix, i)
        for j in range(i+1, rangeMatrix):
            upperMatrix[i][j] = (A[i][j] - summU(upperMatrix, lowerMatrix, i, j)) / lowerMatrix[i][i]
            lowerMatrix[j][i] = (A[i][j] - summL(upperMatrix, lowerMatrix, i, j)) / upperMatrix[i][i]
    duplaDoolittle = [lowerMatrix,upperMatrix]
    return duplaDoolittle

In [5]:
#Método visto en la práctica
#Utiliza las herramientas del módulo numpy
#-------------------------------------------------------------------------------------------------------------------------------#

def factorizarDoolittle(A):
    n = np.shape(A)[0]
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    for i in range (0,n):
        L[i,i] = 1
        U[i,i] = A[i,i]     
        for k in range(0,i):
            U[i,i] -= L[i,k]*U[k,i]          
        for j in range (i+1,n):         
            U[i,j] = A[i,j]             
            L[j,i] = A[j,i]       
            for k in range(0,i):           
                U[i,j] -= L[i,k]*U[k,j]                  
                L[j,i] -= L[j,k]*U[k,i]                
            U[i,j] = U[i,j]/L[i,i] 
            L[j,i] = L[j,i]/U[i,i]               
    return L,U

In [6]:
#Metodo Catedra para realizar sustitución directa e indirecta una vez obtenidos la factorización LU
#-------------------------------------------------------------------------------------------------------------------------#

def sustDirecta(L, B):
    n = np.shape(L)[0]
    Y = np.zeros(n)    
    for i in range(0,n):
        Y[i] = B[i]    
        for j in range (0,i):
            Y[i] += -L[i,j]*Y[j]            
        Y[i] = Y[i] / L[i,i]         
    return Y

def sustInversa(U, Y):
    n = np.shape(U)[0]
    X = np.zeros(n) 
    for i in reversed(range(0,n)):
        X[i] = Y[i]                      
        for j in range (i+1,n):
            X[i] += -U[i,j]*X[j]
        X[i] = X[i] / U[i,i]        
    return X

In [7]:
#Definimos los vectores del sistema para utilizarlos en el método programado por el grupo
#-------------------------------------------------------------------------------------------------------------------------#

matrizComposicion = [[0.83, 0, 0, 0.17], [0, 0.61, 0, 0.39], [0, 0, 0.90, 0.10], [0.55, 0.22, 0, 0.20]]
vectorSolucion = [56000, 21000, 2000, 21000]

In [8]:
#Definimos los arrays en numpy para utilizar la función vista en la práctica
#-------------------------------------------------------------------------------------------------------------------------#

A = np.array([[0.83, 0, 0, 0.55],
       [0, 0.61, 0, 0.22],
       [0, 0, 0.9, 0],
       [0.17, 0.39, 0.1, 0.20]])

B = np.array([56000, 21000, 2000, 21000])

In [9]:
#Llamado a función del método programado por el grupo
#-------------------------------------------------------------------------------------------------------------------------#

descomposicionLU = doolittle(matrizComposicion)
L = np.matrix(descomposicionLU[0])
U = np.matrix(descomposicionLU[1])

print("La diagonal inferior es: \n\n", L)
print("\nLa diagonal superior es: \n\n", U)
print("\nL * U: \n\n", np.dot(L,U))

Y = sustDirecta(L, vectorSolucion)
X = sustInversa(U, Y)

print("El resultado es: \n\n", X)
#pd.DataFrame({ 'Compuestos': Compuestos, 'F1': F1_composicion, 'F2': F2_composicion, 'F3': F3_composicion, 'F4': F4_composicion, 'P': P_composicion })

La diagonal inferior es: 

 [[1.         0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.20481928 0.63934426 0.11111111 1.        ]]

La diagonal superior es: 

 [[ 0.83        0.          0.          0.17      ]
 [ 0.          0.61        0.          0.39      ]
 [ 0.          0.          0.9         0.1       ]
 [ 0.          0.          0.         -0.09527465]]

L * U: 

 [[0.83 0.   0.   0.17]
 [0.   0.61 0.   0.39]
 [0.   0.   0.9  0.1 ]
 [0.17 0.39 0.1  0.2 ]]
El resultado es: 

 [58616.38465188  6790.00686418 -2580.65407774 43225.88669962]


In [10]:
#Llamado a función del método visto en la práctica
#-------------------------------------------------------------------------------------------------------------------------#

descomposicionLU = factorizarDoolittle(A)
L = descomposicionLU[0]
U = descomposicionLU[1]

print("La diagonal inferior es: \n\n", L)
print("\nLa diagonal superior es: \n\n", U)
print("\nL * U: \n\n",np.dot(L,U))

Y = sustDirecta(L, vectorSolucion)
X = sustInversa(U, Y)

print("\nEl resultado es: ",X)
#pd.DataFrame({ 'Compuestos': Compuestos, 'F1': F1_composicion, 'F2': F2_composicion, 'F3': F3_composicion, 'F4': F4_composicion, 'P': P_composicion })

La diagonal inferior es: 

 [[1.         0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.20481928 0.63934426 0.11111111 1.        ]]

La diagonal superior es: 

 [[ 0.83        0.          0.          0.55      ]
 [ 0.          0.61        0.          0.22      ]
 [ 0.          0.          0.9         0.        ]
 [ 0.          0.          0.         -0.05330634]]

L * U: 

 [[0.83 0.   0.   0.55]
 [0.   0.61 0.   0.22]
 [0.   0.   0.9  0.  ]
 [0.17 0.39 0.1  0.2 ]]

El resultado es:  [16274.94328965  6562.75601994  2222.22222222 77257.8128538 ]


In [11]:
#Condicion de matriz y residuo
#-------------------------------------------------------------------------------------------------------------------------#

R1 = B - np.dot(A, X)
rel0 = np.linalg.norm(R1.transpose(),np.inf)/np.linalg.norm(B.transpose(),np.inf)

Knumpy = np.linalg.cond(A,np.inf)

print("Residuo del resultado: ",rel0)
print("\nNumero de condicion de la matriz: ", Knumpy)

Residuo del resultado:  0.0

Numero de condicion de la matriz:  50.61835068608196


In [12]:
#A modo de tener más información sobre los resultados obtenidos, utilizamos eliminación gaussiana
#El principal objetivo de este paso extra es analizar la matriz que obtenemos luego de operar
#-------------------------------------------------------------------------------------------------------------------------#

def matriz_triangular_por_eliminacion_gaussiana(matriz):
	r=matriz
	for i in range(len(matriz)-1):
		for k in range(len(matriz)-1,i,-1):
			if abs(r[k][i])>abs(r[i][i]):
				r[k],r[i]=r[i],r[k]
			if r[k][i]==r[i][i]==0:
				return 'error'
			p=r[k][i]/r[i][i]
			for j in range(len(matriz[0])):
					r[k][j]=r[k][j]-p*r[i][j]
	return r

In [13]:
#Llamado a función del método de eliminación Gaussiana
#-------------------------------------------------------------------------------------------------------------------------#

gaussA = np.matrix(matriz_triangular_por_eliminacion_gaussiana(matrizComposicion))
print(gaussA)

[[ 0.83        0.          0.          0.17      ]
 [ 0.          0.61        0.          0.39      ]
 [ 0.          0.          0.9         0.1       ]
 [ 0.          0.          0.         -0.05330634]]


#### Resultados observados luego de resolver el SEL mediante Wolfram Alpha:
<img src="ResultadoWolfram.png" width=400 height=400 />

##### Podemos observar que el método programado por el grupo tuvo problemas para poder encontrar un resultado cercano al obtenido resolviendo el sistema mediante una calculadora como Wolfram Alpha, mientras que el de la cátedra no. En un primer momento consideramos que tal vez la matriz estaba mal condicionada, pero los calculos de condicionamiento de A apuntaban a lo contrario. También pensamos en utilizar el Método de refinamiento iterativo sobre la solución obtenida por la función del grupo, pero A no cumple las condicones requeridas para poder utilizarlo, ya que A no estaba mal condicionada en el primer lugar.